# Exploring Data from the API

In [70]:
import pandas as pd
import requests as req

from settings import TOKEN

In [2]:
HEADER = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:78.0) Gecko/20100101 Firefox/78.0",
    "Accept-Language": "en-US;q=0.7,es-MX;q=0.3",
    "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
    "Origin": "https://developer.riotgames.com",
    "X-Riot-Token": TOKEN
    }

## Get summoners per league

In [71]:
def get_entries(queue: str, tier: str, division: str, pages: int):
    
    base_url = f"https://euw1.api.riotgames.com/lol/league/v4/entries/{queue}/{tier}/{division}?page={pages}" 
    #RANKED_SOLO_5x5/DIAMOND/I?page=1"
    res = req.get(base_url, headers=HEADER)
    
    return res.json()

In [4]:
entries =  get_entries("RANKED_SOLO_5x5", "DIAMOND", "I", 1)

In [47]:
entries[0]

{'leagueId': 'ade103fb-f7fb-4ca5-bbe1-c3a95b4a219f',
 'queueType': 'RANKED_SOLO_5x5',
 'tier': 'DIAMOND',
 'rank': 'I',
 'summonerId': 'rvTHik0BLi0tKf0oBNH1LB-MrXNHgTYTIJp_xM9Jl22xfFpp',
 'summonerName': 'Meiran',
 'leaguePoints': 90,
 'wins': 200,
 'losses': 187,
 'veteran': False,
 'inactive': False,
 'freshBlood': False,
 'hotStreak': True}

### We are interested on the following columns

- summonerId
- summonerName
- tier
- rank
- leaguePoints
- wins
- loses

**summonerId** is useful to get extra account information

## Get summoners account ids

In [72]:
def get_acc_info(s_id: str):
    
    base_url = f"https://euw1.api.riotgames.com/lol/summoner/v4/summoners/{s_id}" 
    res = req.get(base_url, headers=HEADER)
    
    return res.json()

In [7]:
acc_info = get_acc_info(entries[0]["summonerId"])

In [8]:
acc_info

{'id': 'rvTHik0BLi0tKf0oBNH1LB-MrXNHgTYTIJp_xM9Jl22xfFpp',
 'accountId': 'Xqb-V8M3R76oVcB07Xl9HIW1aQQtlFBdIc0SUN4QJMCvVpg',
 'puuid': '7d4EK5NVYbakMY9VXZ6DSUWMkTp_eHYL3phhtSVPR5OkgqQBM0NrMM8iQej9Ka2fSDjRMR_4X3kggQ',
 'name': 'Meiran',
 'profileIconId': 4568,
 'revisionDate': 1594471446000,
 'summonerLevel': 229}

**accountId** is useful to get the player's match history

## Get match history by summoner

- needs accountId
- needs queue id (420 for 5v5 Ranked Solo Games)
- needs begin time and end time 1593561600 first of july
- needs number of games (we multiply it by 100 to set the beginIndex)

In [73]:
def get_matchlist(a_id: str, begin_time = 1593561600, queue_id = 420):
    base_url = f"https://euw1.api.riotgames.com/lol/match/v4/matchlists/by-account/{a_id}?queue={queue_id}" 
    res = req.get(base_url, headers=HEADER)
    
    return res.json()

In [10]:
matchlist = get_matchlist("Xqb-V8M3R76oVcB07Xl9HIW1aQQtlFBdIc0SUN4QJMCvVpg")

In [11]:
matchlist["matches"][0]

{'platformId': 'EUW1',
 'gameId': 4671787510,
 'champion': 64,
 'queue': 420,
 'season': 13,
 'timestamp': 1592764299945,
 'role': 'NONE',
 'lane': 'JUNGLE'}

**gameId** can be used to retrieve all the data from a specific match

## Request match data with match id

In [74]:
def get_match_data(m_id: str):
    base_url = f"https://euw1.api.riotgames.com/lol/match/v4/matches/{m_id}" 
    res = req.get(base_url, headers=HEADER)
    
    return res.json()

In [13]:
match = get_match_data(matchlist["matches"][0]["gameId"])

In [49]:
match.keys()

dict_keys(['gameId', 'platformId', 'gameCreation', 'gameDuration', 'queueId', 'mapId', 'seasonId', 'gameVersion', 'gameMode', 'gameType', 'teams', 'participants', 'participantIdentities'])

### The following keys are useful for us

- gameId
- participantIdentities
    - player
        - summonerId
- participants
    - stats
    - timeline

In [62]:
match["participants"][0]["stats"].keys()

dict_keys(['participantId', 'win', 'item0', 'item1', 'item2', 'item3', 'item4', 'item5', 'item6', 'kills', 'deaths', 'assists', 'largestKillingSpree', 'largestMultiKill', 'killingSprees', 'longestTimeSpentLiving', 'doubleKills', 'tripleKills', 'quadraKills', 'pentaKills', 'unrealKills', 'totalDamageDealt', 'magicDamageDealt', 'physicalDamageDealt', 'trueDamageDealt', 'largestCriticalStrike', 'totalDamageDealtToChampions', 'magicDamageDealtToChampions', 'physicalDamageDealtToChampions', 'trueDamageDealtToChampions', 'totalHeal', 'totalUnitsHealed', 'damageSelfMitigated', 'damageDealtToObjectives', 'damageDealtToTurrets', 'visionScore', 'timeCCingOthers', 'totalDamageTaken', 'magicalDamageTaken', 'physicalDamageTaken', 'trueDamageTaken', 'goldEarned', 'goldSpent', 'turretKills', 'inhibitorKills', 'totalMinionsKilled', 'neutralMinionsKilled', 'neutralMinionsKilledTeamJungle', 'neutralMinionsKilledEnemyJungle', 'totalTimeCrowdControlDealt', 'champLevel', 'visionWardsBoughtInGame', 'sightWa

In [67]:
match["participantIdentities"][0]["player"].keys()

dict_keys(['platformId', 'accountId', 'summonerName', 'summonerId', 'currentPlatformId', 'currentAccountId', 'matchHistoryUri', 'profileIcon'])

### We need to drop unnecesary data in the stats columns

In [36]:
stats = pd.DataFrame([x["stats"] for x in match["participants"]])

In [38]:
stats = stats.drop(columns=[ 'item0',
 'item1',
 'item2',
 'item3',
 'item4',
 'item5','item6', "unrealKills" ,'combatPlayerScore',
 'objectivePlayerScore',
 'totalPlayerScore',
 'totalScoreRank','playerScore0',
 'playerScore1',
 'playerScore2',
 'playerScore3',
 'playerScore4',
 'playerScore5',
 'playerScore6',
 'playerScore7',
 'playerScore8',
 'playerScore9','perk0',
 'perk0Var1',
 'perk0Var2',
 'perk0Var3',
 'perk1',
 'perk1Var1',
 'perk1Var2',
 'perk1Var3',
 'perk2',
 'perk2Var1',
 'perk2Var2',
 'perk2Var3',
 'perk3',
 'perk3Var1',
 'perk3Var2',
 'perk3Var3',
 'perk4',
 'perk4Var1',
 'perk4Var2',
 'perk4Var3',
 'perk5',
 'perk5Var1',
 'perk5Var2',
 'perk5Var3',
 'perkPrimaryStyle',
 'perkSubStyle',
 'statPerk0',
 'statPerk1',
 'statPerk2'])